# SQL Queries

In [1]:
import psycopg2
import pandas as pd
import os

import warnings
warnings.filterwarnings('ignore')


## Establishing a connnection with a database

We actually need to establish a connection with our database vendor, in this case PostgreSQL, which will allow us to run queries against our database. Postgres is a bit of software that we need to find and connect to. If this is running locally (as in on our computers then we can use our machine's local IP address and just find the port that postgres is running on).

If we are running locally, our connection parameters will look something like the following:

```python
psycopg2.connect(database="databasename", 
                 user="postgres", 
                 password="somepassword",
                 host="127.0.0.1",
                 port="5432")
```

However, this information is sensitive and should never be exposed for security purposes. It is for this reason that this notebook is not hard-coding these values in. Instead, we are setting these values as part of the OS environment and reading them in.

In [2]:
dbname = os.environ.get("DB_NAME")
dbuser = os.environ.get("DB_USER")
dbpass = os.environ.get("DB_PASSWORD")
dbhost = os.environ.get("DB_HOST")
dbport = os.environ.get("DB_PORT")

conn = psycopg2.connect(database=dbname, 
                        user=dbuser, 
                        password=dbpass,
                        host=dbhost,
                        port=dbport)

### A sample query using `psycopg2`

When establishing and familiarizing yourself with a database, it is always best to perform the *hello, world* of SQL...

```sql
SELECT * 
FROM table_name;
```

In [3]:
# a cursor just allows us to execute queries against a database connection
cur = conn.cursor()
cur.execute("SELECT * FROM albums;") 
results = cur.fetchall()

# iterate through the list of tuples
for row in results: 
    print(row)

(1, 'Valentine', 'Snail Mail', datetime.date(2021, 11, 5), 'indie rock')
(2, 'Sweetener', 'Ariana Grande', datetime.date(2018, 8, 17), 'pop')
(3, 'To Pimp A Butterfly', 'Kendrick Lamar', datetime.date(2015, 3, 16), 'hip hop')
(4, 'Bury Me At Makeout Creek', 'Mitski', datetime.date(2014, 11, 11), 'indie rock')
(5, 'Blue', 'Joni Mitchell', datetime.date(1971, 6, 22), 'folk')
(6, 'channel ORANGE', 'Frank Ocean', datetime.date(2012, 7, 10), 'R&B')
(7, 'Changes', 'Justin Bieber', datetime.date(2020, 2, 14), 'pop')
(8, 'Illinoise', 'Sufjan Stevens', datetime.date(2005, 7, 4), 'folk')
(9, 'Sun Giant', 'Fleet Foxes', datetime.date(2008, 4, 8), 'folk')
(10, 'Sounds of Silence', 'Simon & Garfunkle', datetime.date(1966, 1, 17), 'folk')
(11, 'Lemonade', 'Beyoncé', datetime.date(2016, 4, 23), 'R&B')
(12, 'at last!', 'Etta James', datetime.date(1960, 11, 15), 'R&B')
(13, "I'm Wide Awake, It's Morning", 'Bright Eyes', datetime.date(2005, 1, 25), 'folk')
(14, 'Laurel Hell', 'Mitski', datetime.date(202

## Passing the connection to `pandas`

From here on out in this notebook we are going to pass the query and the connection to `pandas`'s `read_sql_query` method for formatting purposes. DataFrames just make the output a bit cleaner.

Let's try it out with the same query.

In [5]:
pd.read_sql_query("SELECT * FROM albums;", conn)

id                                         album_name        artist_name  \
0    1                                          Valentine         Snail Mail   
1    2                                          Sweetener      Ariana Grande   
2    3                                To Pimp A Butterfly     Kendrick Lamar   
3    4                           Bury Me At Makeout Creek             Mitski   
4    5                                               Blue      Joni Mitchell   
5    6                                     channel ORANGE        Frank Ocean   
6    7                                            Changes      Justin Bieber   
7    8                                          Illinoise     Sufjan Stevens   
8    9                                          Sun Giant        Fleet Foxes   
9   10                                  Sounds of Silence  Simon & Garfunkle   
10  11                                           Lemonade            Beyoncé   
11  12                                           at last!         Etta James   
12  13                       I'm Wide Awake, It's Morning        Bright Eyes   
13  14                                        Laurel Hell             Mitski   
14  15                                        Fleet Foxes        Fleet Foxes   
15  16  Village Full of Medical Professionals and a We...         James Bain   

   release_date       genre  
0    2021-11-05  indie rock  
1    2018-08-17         pop  
2    2015-03-16     hip hop  
3    2014-11-11  indie rock  
4    1971-06-22        folk  
5    2012-07-10         R&B  
6    2020-02-14         pop  
7    2005-07-04        folk  
8    2008-04-08        folk  
9    1966-01-17        folk  
10   2016-04-23         R&B  
11   1960-11-15         R&B  
12   2005-01-25        folk  
13   2022-02-05  indie rock  
14   2008-06-03        folk  
15   2025-10-11  indie rock

# Our Exercises for Today

### 1. List the total number of songs

In [18]:
query = """
SELECT COUNT(*) AS num_songs 
FROM songs;
"""

pd.read_sql_query(query, conn)

num_songs
0        186

### 2. List the songs with a rating over 9

In [13]:
query = """
SELECT * 
FROM songs
WHERE rating > 9;
"""

pd.read_sql_query(query, conn)

id  album_id  track_number  \
0    31         3             6   
1    32         3             7   
2    59         5             8   
3    63         6             2   
4    68         6             7   
5    71         6            10   
6    75         6            14   
7    76         6            15   
8    77         6            16   
9    99         8             4   
10  104         8             9   
11  105         8            10   
12  110         8            15   
13  111         8            16   
14  121         9             4   
15  147        12             3   
16  148        12             4   
17  151        12             7   
18  153        12             9   
19  158        13             4   
20  160        13             6   
21  169        14             5   
22  170        14             6   
23  185        15            10   

                                            song_name  rating        run_time  
0                                                   u     9.5 0 days 00:04:28  
1                                             Alright     9.5 0 days 00:03:39  
2                                               River    10.0 0 days 00:04:04  
3                                    Thinkin Bout You     9.5 0 days 00:03:20  
4                                     Super Rich Kids     9.5 0 days 00:05:04  
5                                            Pyramids     9.5 0 days 00:09:52  
6                                        Bad Religion     9.5 0 days 00:02:55  
7                                         Pink Matter     9.5 0 days 00:04:28  
8                                        Forrest Gump     9.5 0 days 00:03:14  
9                                John Wayne Gacy, Jr.     9.5 0 days 00:03:19  
10                              Go! Chicago! Go! Yeah     9.5 0 days 00:06:04  
11                                Casimir Pulaski Day    10.0 0 days 00:05:53  
12  The Predatory Wasp of the Palisades Is Out to ...     9.5 0 days 00:05:23  
13  They Are Night Zombies!! They Are Neighbors!! ...     9.5 0 days 00:05:09  
14                                            Mykonos     9.5 0 days 00:04:35  
15                                        Trust in Me     9.5 0 days 00:03:01  
16                              A Sunday Kind of Love    10.0 0 days 00:03:18  
17                                            At Last    10.0 0 days 00:03:02  
18                                     Stormy Weather     9.5 0 days 00:03:10  
19                                                Lua     9.5 0 days 00:04:30  
20                               First Day of My Life     9.5 0 days 00:03:08  
21                                     Heat Lightning     9.5 0 days 00:02:51  
22                              The Only Heartbreaker     9.5 0 days 00:03:04  
23                               Blue Ridge Mountains     9.5 0 days 00:04:25

### 3. List the albums that have genre of "folk" arranged alphabetically by artist and then by release date newest to oldest.

In [24]:
query = """
SELECT * 
FROM albums
WHERE genre = 'folk'
ORDER BY artist_name, release_date DESC;
"""

pd.read_sql_query(query, conn)

id                    album_name        artist_name release_date genre
0  13  I'm Wide Awake, It's Morning        Bright Eyes   2005-01-25  folk
1  15                   Fleet Foxes        Fleet Foxes   2008-06-03  folk
2   9                     Sun Giant        Fleet Foxes   2008-04-08  folk
3   5                          Blue      Joni Mitchell   1971-06-22  folk
4  10             Sounds of Silence  Simon & Garfunkle   1966-01-17  folk
5   8                     Illinoise     Sufjan Stevens   2005-07-04  folk

### 4. List all of the genres, don't repeat any.

In [29]:
query = """
SELECT DISTINCT genre 
FROM albums;
"""

pd.read_sql_query(query, conn)

genre
0        folk
1         pop
2         R&B
3     hip hop
4  indie rock

## 5. List all of the albums along with their songs.

In [31]:
query = """
SELECT * 
FROM albums
JOIN songs ON albums.id = songs.album_id;
"""

pd.read_sql_query(query, conn)

id   album_name  artist_name release_date       genre   id  album_id  \
0     1    Valentine   Snail Mail   2021-11-05  indie rock    1         1   
1     1    Valentine   Snail Mail   2021-11-05  indie rock    2         1   
2     1    Valentine   Snail Mail   2021-11-05  indie rock    3         1   
3     1    Valentine   Snail Mail   2021-11-05  indie rock    4         1   
4     1    Valentine   Snail Mail   2021-11-05  indie rock    5         1   
..   ..          ...          ...          ...         ...  ...       ...   
181  15  Fleet Foxes  Fleet Foxes   2008-06-03        folk  181        15   
182  15  Fleet Foxes  Fleet Foxes   2008-06-03        folk  182        15   
183  15  Fleet Foxes  Fleet Foxes   2008-06-03        folk  183        15   
184  15  Fleet Foxes  Fleet Foxes   2008-06-03        folk  185        15   
185  15  Fleet Foxes  Fleet Foxes   2008-06-03        folk  186        15   

     track_number             song_name  rating        run_time  
0               1             Valentine     8.0 0 days 00:03:16  
1               2          Ben Franklin     8.5 0 days 00:03:02  
2               3              Headlock     7.5 0 days 00:03:12  
3               4            Light Blue     7.0 0 days 00:02:34  
4               5     Forever (Sailing)     8.0 0 days 00:04:18  
..            ...                   ...     ...             ...  
181             6   He Doesn't Know Why     8.0 0 days 00:03:20  
182             7   Heard Them Stirring     7.5 0 days 00:03:02  
183             8        Your Protector     8.0 0 days 00:04:09  
184            10  Blue Ridge Mountains     9.5 0 days 00:04:25  
185            11          Oliver James     7.5 0 days 00:03:23  

[186 rows x 11 columns]

## 6. List all of the albums along with how many songs each album has.

In [35]:
query = """
SELECT album_name, COUNT(songs.id) AS num_songs
FROM albums
JOIN songs ON albums.id = songs.album_id
GROUP BY album_name;
"""

pd.read_sql_query(query, conn)

album_name  num_songs
0                    Laurel Hell         11
1                      Sweetener         15
2                    Fleet Foxes         11
3              Sounds of Silence         11
4       Bury Me At Makeout Creek         10
5                           Blue         10
6                      Valentine         10
7                 channel ORANGE         17
8   I'm Wide Awake, It's Morning         10
9                      Sun Giant          5
10                       Changes         17
11                      at last!         10
12           To Pimp A Butterfly         16
13                     Illinoise         22
14                      Lemonade         11

## 7. List all of the albums with more than 10 songs.

In [38]:
query = """
SELECT album_name, COUNT(songs.id) AS num_songs
FROM albums
JOIN songs ON albums.id = songs.album_id
GROUP BY album_name
HAVING COUNT(songs.id) > 10;

"""

pd.read_sql_query(query, conn)

album_name  num_songs
0          Laurel Hell         11
1            Sweetener         15
2          Fleet Foxes         11
3    Sounds of Silence         11
4       channel ORANGE         17
5              Changes         17
6  To Pimp A Butterfly         16
7            Illinoise         22
8             Lemonade         11

## 8. List ALL albums along with their songs

In [45]:
query = """
SELECT * 
FROM albums
LEFT JOIN songs ON albums.id = songs.album_id;
"""

pd.read_sql_query(query, conn)

id                                         album_name  artist_name  \
0     1                                          Valentine   Snail Mail   
1     1                                          Valentine   Snail Mail   
2     1                                          Valentine   Snail Mail   
3     1                                          Valentine   Snail Mail   
4     1                                          Valentine   Snail Mail   
..   ..                                                ...          ...   
182  15                                        Fleet Foxes  Fleet Foxes   
183  15                                        Fleet Foxes  Fleet Foxes   
184  15                                        Fleet Foxes  Fleet Foxes   
185  15                                        Fleet Foxes  Fleet Foxes   
186  16  Village Full of Medical Professionals and a We...   James Bain   

    release_date       genre     id  album_id  track_number  \
0     2021-11-05  indie rock    1.0       1.0           1.0   
1     2021-11-05  indie rock    2.0       1.0           2.0   
2     2021-11-05  indie rock    3.0       1.0           3.0   
3     2021-11-05  indie rock    4.0       1.0           4.0   
4     2021-11-05  indie rock    5.0       1.0           5.0   
..           ...         ...    ...       ...           ...   
182   2008-06-03        folk  182.0      15.0           7.0   
183   2008-06-03        folk  183.0      15.0           8.0   
184   2008-06-03        folk  185.0      15.0          10.0   
185   2008-06-03        folk  186.0      15.0          11.0   
186   2025-10-11  indie rock    NaN       NaN           NaN   

                song_name  rating        run_time  
0               Valentine     8.0 0 days 00:03:16  
1            Ben Franklin     8.5 0 days 00:03:02  
2                Headlock     7.5 0 days 00:03:12  
3              Light Blue     7.0 0 days 00:02:34  
4       Forever (Sailing)     8.0 0 days 00:04:18  
..                    ...     ...             ...  
182   Heard Them Stirring     7.5 0 days 00:03:02  
183        Your Protector     8.0 0 days 00:04:09  
184  Blue Ridge Mountains     9.5 0 days 00:04:25  
185          Oliver James     7.5 0 days 00:03:23  
186                  None     NaN             NaT  

[187 rows x 11 columns]

## 9. List albums along with their average song rating.

In [50]:
query = """
SElECT album_name, artist_name, AVG(rating)
FROM albums
JOIN songs ON albums.id = songs.album_id
GROUP BY album_name, artist_name;

"""

pd.read_sql_query(query, conn)

album_name        artist_name       avg
0                      Sun Giant        Fleet Foxes  8.800000
1                           Blue      Joni Mitchell  8.350000
2                      Sweetener      Ariana Grande  7.066667
3                      Illinoise     Sufjan Stevens  8.368421
4   I'm Wide Awake, It's Morning        Bright Eyes  8.500000
5                       Lemonade            Beyoncé  8.409091
6                       at last!         Etta James  9.150000
7       Bury Me At Makeout Creek             Mitski  8.200000
8                 channel ORANGE        Frank Ocean  8.852941
9            To Pimp A Butterfly     Kendrick Lamar  8.375000
10                   Fleet Foxes        Fleet Foxes  8.181818
11                       Changes      Justin Bieber  6.000000
12                     Valentine         Snail Mail  7.750000
13                   Laurel Hell             Mitski  8.409091
14             Sounds of Silence  Simon & Garfunkle  8.500000

### BONUS: List albums and songs with ratings higher than their albums average.

In [58]:
query = """
SELECT album_name, 
    artist_name, 
    song_name,
    rating,
    (SELECT AVG(rating) FROM songs WHERE songs.album_id = albums.id) AS avg_album
FROM albums
JOIN songs on albums.id = songs.album_id
WHERE rating > (SELECT AVG(rating) FROM songs WHERE songs.album_id = albums.id);

"""

pd.read_sql_query(query, conn)

album_name  artist_name                    song_name  rating
0     Valentine   Snail Mail                    Valentine     8.0
1     Valentine   Snail Mail                 Ben Franklin     8.5
2     Valentine   Snail Mail            Forever (Sailing)     8.0
3     Valentine   Snail Mail                      Madonna     9.0
4     Valentine   Snail Mail                          Mia     8.0
..          ...          ...                          ...     ...
77  Laurel Hell       Mitski        The Only Heartbreaker     9.5
78  Laurel Hell       Mitski              That's Our Lamp     8.5
79  Fleet Foxes  Fleet Foxes          White Winter Hymnal     9.0
80  Fleet Foxes  Fleet Foxes  Tiger Mountain Peasant Song     8.5
81  Fleet Foxes  Fleet Foxes         Blue Ridge Mountains     9.5

[82 rows x 4 columns]